In [8]:
%pylab inline
 
import warnings
warnings.filterwarnings("ignore")
 
import datetime, time
 
import pytalib as ta
 
import numpy as np
import pandas as pd
from pandas import DataFrame
import pandas.io.sql as pdsql
 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.font_manager as font_manager
import seaborn as sns
 
import mysql.connector

Populating the interactive namespace from numpy and matplotlib


ModuleNotFoundError: No module named 'mysql'

In [ ]:
#맑은고딕체
sns.set(style="whitegrid", font="Malgun Gothic", font_scale=1.5)
fp = font_manager.FontProperties(fname="C:\\WINDOWS\\Fonts\\malgun.TTF", size=15)
 
def comma_volume(x, pos=None):
    s = '{:0,d}K'.format(int(x/1000))
    return s
 
def comma_price(x, pos=None):
    try:
        s = '{:0,d}'.format(int(x))
    except Exception as e:
        s = 'NaN'
    return s
 
def comma_percent(x, pos=None):
    s = '{:+.2f}'.format(x)
    return s
 

In [ ]:

major_date_formatter = dates.DateFormatter('%Y-%m-%d')
minor_date_formatter = dates.DateFormatter('%m')
price_formatter = ticker.FuncFormatter(comma_price)
volume_formatter = ticker.FuncFormatter(comma_volume)
percent_formatter = ticker.FuncFormatter(comma_percent)
 
MySQL_POOL_SIZE = 2
 
데이타베이스_설정값 = {
    'host': '127.0.0.1', 
    'database': 'mymoneybot', 
    'user': 'mymoneybot', 
    'password': 'mymoneybot', 
    'raise_on_warnings': True,
}
 
class NumpyMySQLConverter(mysql.connector.conversion.MySQLConverter):
    """ A mysql.connector Converter that handles Numpy types """
 
    def _float32_to_mysql(self, value):
        return float(value)
 
    def _float64_to_mysql(self, value):
        return float(value)
 
    def _int32_to_mysql(self, value):
        return int(value)
 
    def _int64_to_mysql(self, value):
        return int(value)
 
    def _timestamp_to_mysql(self, value):
        return value.to_datetime()
 
def mysqlconn():    
    conn = mysql.connector.connect(pool_name="stockpool", pool_size=MySQL_POOL_SIZE, **데이타베이스_설정값)
    conn.set_converter_class(NumpyMySQLConverter)
    return conn
 

In [3]:

# 데이타를 기간에 맞게 잘라내는 함수
def 기간(dataframe, 시작기간=None, 종료기간=None):
    df = dataframe.copy()
 
    if (시작기간 is None) and (종료기간 is None):
        pass
    elif (시작기간 is None) and not(종료기간 is None):
        df = df[:종료기간]
    elif not(시작기간 is None) and (종료기간 is None):
        df = df[시작기간:]
    elif not(시작기간 is None) and not(종료기간 is None):
        df = df[시작기간:종료기간]
 
    return df
 

In [4]:
 
def get_price(code, 시작일자=None, 종료일자=None):
    """지정한 종목의 가격/거래량 정보를 읽어 가공한다"""
    
    if 시작일자 == None and 종료일자 == None:
        query = """
        SELECT 일자, 시가, 고가, 저가, 종가, 거래량
        FROM 일별주가
        WHERE 종목코드='%s'
        ORDER BY 일자 ASC
        """ % (code)
    if 시작일자 != None and 종료일자 == None:
        query = """
        SELECT 일자, 시가, 고가, 저가, 종가, 거래량
        FROM 일별주가
        WHERE 종목코드='%s' AND 일자 >= '%s'
        ORDER BY 일자 ASC
        """ % (code, 시작일자)
    if 시작일자 == None and 종료일자 != None:
        query = """
        SELECT 일자, 시가, 고가, 저가, 종가, 거래량
        FROM 일별주가
        WHERE 종목코드='%s' AND 일자 <= '%s'
        ORDER BY 일자 ASC
        """ % (code, 종료일자)
    if 시작일자 != None and 종료일자 != None:
        query = """
        SELECT 일자, 시가, 고가, 저가, 종가, 거래량
        FROM 일별주가
        WHERE 종목코드='%s' AND 일자 BETWEEN '%s' AND '%s'
        ORDER BY 일자 ASC
        """ % (code, 시작일자, 종료일자)
 
    conn = mysqlconn()
    df = pdsql.read_sql_query(query, con=conn)
    conn.close()
    
    df.fillna(0, inplace=True)
    df.set_index('일자', inplace=True)
    

In [5]:
   #
    # 
    
    df['등락률'] = np.log(df['종가'] / df['종가'].shift(1)) # 어제 종가로 오늘종가를 나눈 것 얼마나 변동이 있었는지 알기 위해(오차가 심해져서 로그를 씌워서코드)
    df['변동성'] = np.sqrt(df['등락률'].rolling(window=30).std()) * 100
 #등락률을 가지고 30일간의 표준편차에 루트를 씌운 것 (30일간의 변동성에 대한 것)
    ###
    
    df.dropna(inplace=True)
    return df
 
시작일자='2015-01-01'
종료일자='2017-12-31'
 
df = get_price(code='069500', 시작일자=시작일자, 종료일자=종료일자) #code='069500' ==코덱스200(ETF)의 지수
#단일종목에 한해 빅스지수를 만든 것  /  
 
df['이동평균'] = df['종가'].rolling(window=30).mean()
df['변동성이동평균'] = df['변동성'].rolling(window=30).mean()
df['변동차'] = df['변동성'] - df['변동성이동평균']
 
df['mean'] = df['변동차'].rolling(window=15).mean()
df['std'] = df['변동차'].rolling(window=15).std()
df['signal'] = (df['변동차'] - df['mean']) / df['std']
 
df.dropna(inplace=True)
 
fig, axes = plt.subplots(3, 1, figsize=(20,15), sharex=True)
ax = df[['종가','이동평균']].plot(ax=axes[0], color="black")
ax.xaxis.set_major_formatter(major_date_formatter)
ax.yaxis.set_major_formatter(price_formatter)
ax.set_xlabel('', fontproperties=fp)
 
ax = df[['변동성','변동성이동평균']].plot(ax=axes[1])
ax.xaxis.set_major_formatter(major_date_formatter)

IndentationError: unexpected indent (Temp/ipykernel_2344/1741953321.py, line 4)

In [ ]:
# ax.yaxis.set_major_formatter(price_formatter)
ax.set_xlabel('', fontproperties=fp)
 
ax = df['signal'].plot(ax=axes[2])
ax.xaxis.set_major_formatter(major_date_formatter)
ax.yaxis.set_major_formatter(price_formatter)
ax.axhline(2, linestyle='--', color='red')
ax.axhline(-2, linestyle='--', color='blue')
ax.set_xlabel('', fontproperties=fp)
plt.show()


In [ ]:
#그래프는 
#첫번째 - 코덱스200의 지수와 이동평균
#두번째 - 변동성과 변동성을 이동평균을 낸 것의 그래프 (돌파할때의 신호를 삼기 위해) - 변동성이 떨어지면 주가는 상승하는 형태를 보임
#세번째 - 평균값에서 얼마나 벌어졌는지 알고싶은것(정규화) - 0을 기준으로 얼마나 벌어졌는지 - 변동성 여부 확인하는 지수